# Azure MVAD Service Guide - Part II (Python API)

What follows is a guide via example on how to use Azure's MVAD Service. Make sure to check the service's documentation [here](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/overview), as well as the code provided [here](https://github.com/Azure-Samples/AnomalyDetector). Note that there are two approaches to using the MVAD Service: via the SDK and via the API. The latter is presented in this notebook.

## Initial Setup

To use the MVAD Service we obviously need to create an [Anomaly Detector subscription](https://azure.microsoft.com/en-us/free/cognitive-services/), as well as create an [Anomaly Detector resource](https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesAnomalyDetector). From the created resource, we need the endpoint's name, as well as the key. For safety, the key can be inserted into an Azure Key Vault and read from there by creating a [secret scope in Databricks](https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes). Additionally, a Storage Account is required, from where the data are drawn to perform anomaly detection. The MVAD Service (Managed Identity) must have the Storage Blob Data Reader role assigned to it in order to be able to draw data from the Storage Account.

For the purposes of the present demonstration, we create a new Container named `test-mvad` in the Storage Account present in the same resource group and send the [sample data](https://github.com/Azure-Samples/AnomalyDetector/blob/master/sampledata/multivariate/multivariate_sample_data.csv) that are going to be used as an example. The MVAD Service accepts data in both multi-table (one `.csv` file for each feature) and single-table (a single `.csv` file with multiple columns, one per feature plus the timestamp columns) forms, but we opt to work with the single-table approach.

In [ ]:
import json
import requests
import time
import pandas as pd

The following library needs to be installed. Note that its installation can be attached to the startup of a specific cluster that runs the present notebook, or to a script that is activated whenever the cluster starts.

In [ ]:
#!pip install --upgrade azure-ai-anomalydetector

## Setup

First, we setup all the necessary variables. Something important to note here is that the endpoint used for the MVAD Service is different compared to the one used in the previous notebook (`/anomalydetector/v1.1` is added to the previous endpoint).

In [ ]:
# Setup the variables
ad_name = "REDACTED"
ENDPOINT = f"https://{ad_name}.cognitiveservices.azure.com/anomalydetector/v1.1"
HEADERS = {"Ocp-Apim-Subscription-Key": dbutils.secrets.get(scope = "anomdet", key = "adkey")}

# If the data is one CSV file, set the dataSchema as `OneTable`
# If the data is multiple CSV files in a folder, set the dataSchema as `MultiTable`.
DATA_SCHEMA="OneTable"

#%%%%%%------------------------

# MVAD-related requests
API_MODEL = "{endpoint}/multivariate/models"
API_MODEL_STATUS = "{endpoint}/multivariate/models/{model_id}"
API_MODEL_BATCH_INFERENCE = "{endpoint}/multivariate/models/{model_id}:detect-batch"
API_RESULTS = "{endpoint}/multivariate/detect-batch/{result_id}"
API_DELETE = "{endpoint}/multivariate/models/{model_id}"

# Data source
# WARNING! Make sure to grant 'Storage Blob Data Reader' role to the Anomaly Detector
# otherwise a FailToAccessBlobURLError occurs.
# Steps:
# [1] Find the storage service
# [2] Go to Access Control(IAM), and select +ADD to Add role assignment.
# [3] Search role of Storage Blob Data Reader, click on it and then select Next.
# [4] Select assign access to Managed identity, and Select Members, then choose the anomaly detector resource, Review + assign.

blobname = "REDACTED"
containername = "test-mvad"
filename = "sample.csv"
DATA_SOURCE_URL = f"https://{blobname}.blob.core.windows.net/{containername}/{filename}"

## Basic Functionalities

We can proceed to see some of the basic functionalities of the MVAD Service.

### Training

The first and perhaps most important one is the training of the model. This is done using the `API_MODEL` API.

In [ ]:
# Parameters for training
SLIDING_WINDOW = 200 # Number of data points used to determine anomalies (default = 200). Equal to the segment used to decide if the next data point is an anomaly
ALIGN_MODE = 'Outer' # How to pick timestamps when the CSVs have different timestamps
FILL_NA = 'Linear' # How to fill missing values - this case does not concern the mock dataset
PADDING = 0 # Only relevant for 'Fixed' FILL_NA

MODEL_NAME = 'sample_model' # Model display name

TRAINING_START_TIME = "2021-02-18T13:50:00Z" # Start time of training/inference
TRAINING_END_TIME = "2021-03-23T10:00:00Z" # End time of training/inference

train_params = {
    "slidingWindow": SLIDING_WINDOW,
    "alignPolicy": {
        "alignMode": ALIGN_MODE,
        "fillNAMethod": FILL_NA, 
        "paddingValue": PADDING
    },
    "dataSource": DATA_SOURCE_URL,
    "dataSchema": DATA_SCHEMA,
    "startTime": TRAINING_START_TIME, 
    "endTime": TRAINING_END_TIME, 
    "displayName": MODEL_NAME
}

# Request to start training the model
train_req = requests.post(API_MODEL.format(endpoint=ENDPOINT), data=json.dumps(train_params), headers=HEADERS)
# Print the following line to get the epochId. There are 100 epochIds in total.
#print(json.loads(train_req.content))
# Get model location
location = train_req.headers['Location']
# The location is used in order to get the model_id
model_id = location[location.rindex('/')+1:]
print(f"The created model's ID is {model_id}.\nCheck the model's status using the next cell.")

The created model's ID is 036c7f2e-e91e-11ed-ba0c-567fe128847f.
Check the model's status using the next cell.


At this point, the new model is being trained behind the shadows. We can define a loop that will perform a check for us and notify us once the model is truly ready. Once it is, we can also get some diagnostics.

In [ ]:
while True:
    # Perform a status request every 20 seconds, to see when the model is ready.
    status_req = requests.get(API_MODEL_STATUS.format(endpoint=ENDPOINT, model_id = model_id), headers=HEADERS)

    res_content = json.loads(status_req.content)
    if res_content['modelInfo']['status'] == 'READY':
        print("The model has been successfully trained.\n")
        break
    else:
        print("The model is still in training mode. Please wait...")
        time.sleep(45)

# Print diagnostics once the model is ready
print('Training Info:\n')
display(pd.DataFrame(res_content['modelInfo']['diagnosticsInfo']['modelState']))
print("\nVariables Info:\n")
display(pd.DataFrame(res_content['modelInfo']['diagnosticsInfo']['variableStates']))
print(f"\nModel status: {res_content['modelInfo']['status']}")

The model is still in training mode. Please wait...
The model is still in training mode. Please wait...
The model is still in training mode. Please wait...
The model is still in training mode. Please wait...
The model has been successfully trained.

Training Info:



epochIds,trainLosses,validationLosses,latenciesInSeconds
10,0.5775568356205311,0.8278930401953871,1.4413204193115234
20,0.5704785206222108,0.6260857462843046,1.3562369346618652
30,0.5613284353832049,0.6929799669569523,1.3472182750701904
40,0.5523706069881362,0.7389550165500463,1.432410478591919
50,0.5777624892736121,0.629393915648275,1.4331724643707275
60,0.5588577846730395,0.6556564748546873,1.4484691619873047
70,0.5595402237959206,0.5841600478454505,1.3884916305541992
80,0.5608177969365247,0.6237323151017482,1.3792550563812256
90,0.5589803169215364,0.6192049300023442,1.413428544998169
100,0.5633129495462137,0.6043745457961276,1.416254997253418



Variables Info:



variable,filledNARatio,effectiveCount,firstTimestamp,lastTimestamp
opticalLFiltered,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
opticalRFiltered,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
pumpPressure,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
rotational,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
vibrationHorizon,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z



Model status: READY


### Inference

The other main functionality is inference. In Azure MVAD there are two types of inference: batch and streaming, where each is self-explanatory. Streaming is oriented towards real-time problems, where each new point is classified as anomalous or not individually. For the purposes of our project, batch inference is the go-to, so we will focus on this in what follows. For more information about Streaming Inference you can refer [here](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/how-to/streaming-inference).

In [ ]:
# Parameters for inference
INFERENCE_START_TIME = "2021-09-09T15:10:00Z"
INFERENCE_END_TIME = "2021-09-14T23:20:00Z"

# topContributorCount: how many contributed variables we care about in the results
inf_params = {
    "dataSource": DATA_SOURCE_URL,
    "topContributorCount": 10,
    "startTime": INFERENCE_START_TIME,
    "endTime": INFERENCE_END_TIME
}

# Request to start inference
inf_req = requests.post(API_MODEL_BATCH_INFERENCE.format(endpoint=ENDPOINT, model_id=model_id), data=json.dumps(inf_params), headers=HEADERS)

# Get the id for the result file
result_id = inf_req.headers["operation-id"]
print(f"A batch inference is triggered with the result id: {result_id}.\nRefer to the next cell for the detection results.")

A batch inference is triggered with the result id: 7aa6c612-e91e-11ed-a041-c676e9936544.
Refer to the next cell for the detection results.


Similarly with the training process, the inference process might also take a while, depending on data types and sizes. We need to wait until the results are ready. Once they are, a different API call needs to be performed to obtain the results.

In [ ]:
# Retrieve the inference result
while True:
    retr_req = requests.get(API_RESULTS.format(endpoint=ENDPOINT, result_id=result_id), headers=HEADERS)
    retr_cont = json.loads(retr_req.content)
    
    if retr_cont['summary']['status'] == 'RUNNING':
        print('The model is not ready yet. Still inferring...')
        time.sleep(15)
    else:
        print("The inference results are ready.\n")
        break
        
print('\n------------------ Summary ------------------ ')
display(pd.DataFrame(retr_cont['summary']['variableStates']))

The model is not ready yet. Still inferring...
The inference results are ready.


------------------ Summary ------------------ 


variable,filledNARatio,effectiveCount,firstTimestamp,lastTimestamp
opticalLFiltered,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
opticalRFiltered,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
pumpPressure,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
rotational,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
vibrationHorizon,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z


The results in a DataFrame format can be seen by running the following cell. Of course, a dataframe of results is not the best medium of serving them, which is why we will also proceed with a visualization thereof in the following notebook.

In [ ]:
adf = pd.DataFrame([{'timestamp': x['timestamp'], **x['value']} for x in retr_cont['results']])
adf['timestamp'] = pd.to_datetime(adf['timestamp'], utc=True)
adf

,timestamp,isAnomaly,severity,score,interpretation
0,2021-09-09 15:10:00+00:00,True,0.067947,0.116752,"[{'variable': 'vibrationHorizon', 'contributio..."
1,2021-09-09 15:20:00+00:00,True,0.096815,0.166356,"[{'variable': 'vibrationHorizon', 'contributio..."
2,2021-09-09 15:30:00+00:00,True,0.111642,0.191832,"[{'variable': 'vibrationHorizon', 'contributio..."
3,2021-09-09 15:40:00+00:00,True,0.102313,0.175803,"[{'variable': 'opticalRFiltered', 'contributio..."
4,2021-09-09 15:50:00+00:00,True,0.113587,0.195174,"[{'variable': 'rotational', 'contributionScore..."
...,...,...,...,...,...
765,2021-09-14 22:40:00+00:00,False,0.000000,0.015003,[]
766,2021-09-14 22:50:00+00:00,False,0.000000,0.015003,[]
767,2021-09-14 23:00:00+00:00,False,0.000000,0.015003,[]
768,2021-09-14 23:10:00+00:00,False,0.000000,0.015003,[]


We can see that the results consist of the following fields:

- **isAnomaly**: the main field, indicating whether the given timestamp corresponds to an anomaly or not.
- **severity**: indicates the significance of the anomaly. The higher the severity, the more significant the anomaly is. A new threshold can be assigned so that anomalies are not found simply by checking the boolean `isAnomaly`, but also by checking if their severity crosses said threshold.
- **score**: raw anomaly, helps indicate the degree of abnormality as well. Notice that even non-anomalous points have a non-zero score, which is what makes this metric different from `severity`.
- **interpretation**: The interpretation result corresponds to a list where the contribution of each feature for the inference is given. This contribution is given as a score, as well as in the form of a `~anomalydetector.models.CorrelationChanges` type object. This object is basically a list of other features, for which the correlation between themselves and the studied feature has changed, as detected by the model.

### List available models

Another thing we can do is get a list of the available models.

In [ ]:
# Get a list + info for the existing models
avaib_models = requests.get(API_MODEL.format(endpoint=ENDPOINT), headers=HEADERS)

resp = json.loads(avaib_models.content)

try:
    models_df = pd.DataFrame([{'Model ID': x['modelId'], "Created" : x['createdTime'], "Last Updated" : x['lastUpdatedTime'], "Status" : x['modelInfo']['status'], "Name" : x['modelInfo']['displayName'], "Errors" : x['modelInfo']['errors'] if x['modelInfo']['errors'] else "None", "Variables Count" : len(x['modelInfo']['diagnosticsInfo']['variableStates'])} for x in resp['models']])

    display(models_df)
except IndexError:
    print("There are no models available to show.")

Model ID,Created,Last Updated,Status,Name,Errors,Variables Count
REDACTED,2023-05-02T19:17:43Z,2023-05-02T19:20:14Z,READY,sample_model,None,5
REDACTED,2022-12-13T17:47:32Z,2022-12-13T17:49:27Z,READY,MVADModel,None,3


Here we can see the model we trained on the sample data, as well as an older model.

### Delete model

Using the `API_DELETE` API, we may delete the model we trained from the registry, by providing its id as an argument. For example, we can get the id of the model we just trained and delete it from the registry.

In [ ]:
model_id = '036c7f2e-e91e-11ed-ba0c-567fe128847f' # <- insert string of ModelId to be deleted

delete_res = requests.delete(API_DELETE.format(endpoint=ENDPOINT, model_id=model_id), headers=HEADERS)

If we now attempt to see again the list of available models, we will see that the model we trained has been successfully deleted.

In [ ]:
# Get a list + info for the existing models
avaib_models = requests.get(API_MODEL.format(endpoint=ENDPOINT), headers=HEADERS)

resp = json.loads(avaib_models.content)

try:
    models_df = pd.DataFrame([{'Model ID': x['modelId'], "Created" : x['createdTime'], "Last Updated" : x['lastUpdatedTime'], "Status" : x['modelInfo']['status'], "Name" : x['modelInfo']['displayName'], "Errors" : x['modelInfo']['errors'] if x['modelInfo']['errors'] else "None", "Variables Count" : len(x['modelInfo']['diagnosticsInfo']['variableStates'])} for x in resp['models']])
    
    display(models_df)
except IndexError:
    print("There are no models available to show.")

Model ID,Created,Last Updated,Status,Name,Errors,Variables Count
REDACTED,2022-12-13T17:47:32Z,2022-12-13T17:49:27Z,READY,MVADModel,None,3


This sums up the main functionalities of the MVAD Service using the Python API. Please refer to the previous notebook for how to use the Service using the SDK, as well as the following notebook for visualization examples.